In [1]:
import pandas as pd
import ipyannotations.text
from ipyannotations.controls.togglebuttongroup import ToggleButtonGroup
import IPython.display
import ipywidgets as widgets
import random
from datetime import datetime
import json
import traitlets

In [2]:
# Initialize

data_to_annotate = pd.read_csv('summary_table.csv', index_col=0)

# indices_shuffle = list(range(len(data_to_annotate)))
# random.seed(42)
# random.shuffle(indices_shuffle)
# data_to_annotate = data_to_annotate.iloc[indices_shuffle,:].reset_index()

In [3]:
TEMPLATE = """
#### Claim:

{}

#### Evidence:

{}
"""

class MCQToggleButtonGroup(ToggleButtonGroup):
    def __init__(self, options):
        super().__init__(options=options)

    @traitlets.observe("value")
    def update_toggles(self, change):  # noqa: D001
        old, new = change["old"], change["new"]
        with self.hold_trait_notifications():
            # Enforcing <=1 option selected
            if isinstance(old, list) and isinstance(new, list):
                if len(old) > 0:
                    if len(old) > 1 or (len(new) != 2 and len(new) != 0):
                        raise NotImplementedError(
                            f"Incorrect sizes for MCQToggleButtonGroup.update_toggeles.change: len(old) = {len(old)}, len(new) = {len(new)}"
                        )
    
                    if len(new) == 2:
                        new_value = new[0] if new[1] == old[0] else new[1]
                        for button in self.buttons:
                            button.value = button.description == new_value
                        return
            
            for button in self.buttons:
                button.value = button.description in change["new"]

    def _toggle(self, option: str):
        for i in range(len(self.options)):
            if self.options[idx] == option:
                self.buttons[idx].value = not self.buttons[idx].value
            else:
                self.buttons[idx].value = False

class ClaimEvLabeller(ipyannotations.text.MulticlassLabeller):
    data_conflict_check = traitlets.Bool(False)
    data_conflict_sol = traitlets.List(trait=traitlets.Unicode(), default_value=list())
    data_comment = traitlets.Unicode("")
    
    def __init__(self):
        super().__init__(
            options=['Supports', 'Does not support'],
            allow_freetext=True
        )

        self.class_selector = MCQToggleButtonGroup(options=['Supports', 'Does not support'])
        self.free_text_area = widgets.Textarea(
            placeholder="Explain the reasoning behind your annotation.",
            rows=5,
            layout={"width": "95%"},
        )
        self.conflict_checkbox = widgets.Checkbox(
            value=False,
            description="Conflict among evidence pieces",
            disabled=False,
            indent=False
        )
        self.conflict_solution_button = MCQToggleButtonGroup(options=[
            "The claim is consistent with all evidence pieces (likely by providing a range of acceptable values)",
            "The claim is only consistent with one of the evidence pieces"
        ])

        # Label data is in self.data
        traitlets.link((self, "data"), (self.class_selector, "value"))
        traitlets.link((self, "data_conflict_check"), (self.conflict_checkbox, "value"))
        traitlets.link((self, "data_conflict_sol"), (self.conflict_solution_button, "value"))
        traitlets.link((self, "data_comment"), (self.free_text_area, "value"))
        

        self.save_submitted_button = widgets.Button(description="💾 Save Submitted Records", button_style='success')
        def on_save_button_click(b):
            try:
                with open(f"annotations.json", "r") as f:
                    records = json.load(f)
            except:
                records = []
            records.extend(self.labels)
            with open(f"annotations.json", "w") as f:
                json.dump(records, f, indent=2)
            
            print(f"Saved {len(self.labels)} annotations to annotations.json")
            self.labels = []
        self.save_submitted_button.on_click(on_save_button_click)
        

        self.children = [
            widgets.Box(
                (self.display_widget,),
                layout=widgets.Layout(
                    justify_content="center",
                    padding="2.5% 0",
                    display="flex",
                    width="100%",
                ),
            ),
            self.class_selector,
            self.conflict_checkbox,
            self.conflict_solution_button,
            self.free_text_area,
            widgets.HBox(
                [
                    self.submit_button,
                    self.save_submitted_button
                ],
                layout=widgets.Layout(justify_content="flex-end"),
            ),
            self.event_watcher,
        ]

        self.data_idx = 0
        self.labels = []

        self.display()

    def submit(self, sender=None):
        """The function that gets called by submitting an option.

        This is called by the button / text field elements and shouldn't be
        called directly.

        Parameters
        ----------
        sender : Optional
            The "sender" that invoked this callback. This is ignored.
        """
        
        if hasattr(self, "data") and hasattr(self, "data_conflict_check") and hasattr(self, "data_conflict_sol") and hasattr(self, "data_comment"):
            pass
        else:  # pragma: no cover
            raise NotImplementedError(
                "Submission for this widget doesn't seem to be implemented."
            )

        self.labels.append((
            self.data_idx,
            self.data,
            self.data_conflict_check,
            self.data_conflict_sol,
            self.data_comment
        ))
        self.data_idx += 1
        try:
            self.display()
            for field in ["data", "data_conflict_check", "data_conflict_sol", "data_comment"]:
                if field in self.class_traits():
                    setattr(self, field, self.class_traits()[field].default())
        except KeyError:
            self.display("Finished. Thanks for participating!")
            self.options = []
            self.conflict_checkbox = widgets.HBox([])
            self.conflict_solution_button = widgets.HBox([])
            self.free_text_area = widgets.HBox([])

    def display(self):
        data_point = data_to_annotate.loc[self.data_idx, :]
        super().display(TEMPLATE.format(data_point["formatted_claim"], data_point["formatted_evidence_layout"]))


widget = ClaimEvLabeller()

In [4]:
widget

ClaimEvLabeller(children=(Box(children=(Output(layout=Layout(margin='auto', min_height='50px')),), layout=Layo…

In [5]:
# indices_shuffle[:20]